In [67]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
#!mkdir vision

In [ ]:
# 위노그라드 알고리즘 설정
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'

In [ ]:
!git clone https://github.com/pinktop/ComputerVisionProject1.git

Cloning into 'ComputerVisionProject1'...
remote: Enumerating objects: 1375, done.
remote: Counting objects: 100% (1375/1375), done.
remote: Compressing objects: 100% (1373/1373), done.
remote: Total 1375 (delta 47), reused 1301 (delta 1), pack-reused 0
Receiving objects: 100% (1375/1375), 164.18 MiB | 28.68 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [ ]:
rootPath = './project2/vision/'

In [ ]:
def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser


In [ ]:
imageGenerator = ImageDataGenerator(
    rotation_range=20,
    horizontal_flip=True,
    preprocessing_function = get_random_eraser(v_l=0, v_h=255),
    validation_split=.1
)

In [ ]:
trainGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='training'
)

Found 1157 images belonging to 3 classes.


In [ ]:
validationGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'training_set'),
    target_size=(100, 100),
    subset='validation'
)

Found 127 images belonging to 3 classes.


# 모델 구성

In [ ]:
model = Sequential()
model.add(ResNet50(include_top=True, weights=None, input_shape=(100, 100, 3), classes=3))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 3)                 23593859  
                                                                 
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy', 
    metrics=['acc'],
)

In [101]:
epochs = 150
history = model.fit_generator(
    trainGen, 
    epochs=epochs,
    steps_per_epoch=trainGen.samples / epochs, 
    validation_data=validationGen,
    validation_steps=trainGen.samples / epochs,
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/150
7/7 [==============================] - 48s 6s/step - loss: 0.0765 - acc: 0.9651
Epoch 2/150
7/7 [==============================] - 56s 7s/step - loss: 0.0971 - acc: 0.9180
Epoch 3/150
7/7 [==============================] - 54s 7s/step - loss: 0.0572 - acc: 0.9648
Epoch 4/150
7/7 [==============================] - 48s 6s/step - loss: 0.0835 - acc: 0.9520
Epoch 5/150
7/7 [==============================] - 53s 7s/step - loss: 0.0858 - acc: 0.9414
Epoch 6/150
7/7 [==============================] - 54s 7s/step - loss: 0.0573 - acc: 0.9805
Epoch 7/150
7/7 [==============================] - 54s 7s/step - loss: 0.0761 - acc: 0.9570
Epoch 8/150
7/7 [==============================] - 54s 7s/step - loss: 0.0830 - acc: 0.9414
Epoch 9/150
7/7 [==============================] - 53s 7s/step - loss: 0.0704 - acc: 0.9609
Epoch 10/150
7/7 [==============================] - 53s 7s/step - loss: 0.0560 - acc: 0.9688
Epoch 11/150
7/7 [==============================] - 52s 7s/step - loss: 0.0593 

# 평가

In [115]:
testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(100, 100),
)

Found 15 images belonging to 3 classes.


In [116]:
pred = model.predict(testGen)

In [117]:
img_list = testGen.filenames

In [118]:
pred_list = []
for i in range(len(pred)):
    pred_list.append(np.argmax(pred[i]))

In [119]:
df = pd.DataFrame(list(zip(img_list, pred_list)), columns =['Name', 'pred'])

In [120]:
df.to_csv('submission.csv', index=False)

In [114]:
print("정확도 :", model.evaluate_generator(testGen))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


정확도 : [0.05831601470708847, 1.0]
